In [13]:
%load_ext autoreload
%autoreload 2
import numpy as np
from vflow import Vset, init_args, dict_to_df, perturbation_stats
from functools import partial
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
ind = {}


def top_n_features(X, Y, n, i):
    if i not in ind:
        corr = np.abs(np.apply_along_axis(lambda x: np.corrcoef(x, Y[:, i])[0, 1], 0, X))
        ind[i] = np.argsort(corr[~np.isnan(corr)])[::-1][:n]
    return X[:, ind[i]]


def pca(X, n):
    return PCA(n_components=n, copy=True).fit(X).transform(X)


In [4]:
# load data
data_dir = "./data/fmri/"
X = np.load(data_dir + "fit_feat.npy")
Y = np.load(data_dir + "resp_dat.npy")

In [5]:
np.random.seed(14)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=14)
X_train, X_test, y_train, y_test = init_args((X_train, X_test, y_train, y_test),
                                             names=['X_train', 'X_test', 'y_train', 'y_test'])

# split y_train by voxel and extract top 500 correlated features per voxel
voxel_extract_funcs = [partial(lambda x, y, i: (top_n_features(x, y, 20, i), y[:, i]), i=i) for i in range(20)]
voxel_extract_set = Vset(name='voxel_extract', modules=voxel_extract_funcs, output_matching=True)
X_trains, y_trains = voxel_extract_set(X_train, y_train)
X_tests, y_tests = voxel_extract_set(X_test, y_test)

/Users/rush/anaconda3/envs/vflow/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/rush/anaconda3/envs/vflow/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [6]:
# modeling
modeling_set = Vset(name='modeling', modules=[LassoCV()], module_keys=["Lasso"])
modeling_set.fit(X_trains, y_trains)

In [7]:
preds = modeling_set.predict(X_trains)

hard_metrics_set = Vset(name='hard_metrics', modules=[r2_score, explained_variance_score],
                             module_keys=["R2", "EV"])
hard_metrics = hard_metrics_set.evaluate(y_trains, preds)

In [12]:
df = dict_to_df(hard_metrics)
mean_metrics = perturbation_stats(df, 'voxel_extract', func=['mean'])

mean_metrics

,voxel_extract,mean
0,voxel_extract_0,0.158618
1,voxel_extract_1,0.132072
2,voxel_extract_10,0.054496
3,voxel_extract_11,0.156664
4,voxel_extract_12,0.000000
5,voxel_extract_13,0.069849
6,voxel_extract_14,0.232675
7,voxel_extract_15,0.000000
8,voxel_extract_16,0.057368
9,voxel_extract_17,0.139997
